# Part 1: Format the Canada's online table

In [1]:
import pandas as pd

df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=df[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Get rid of the cell with "Not assigend" for Borough column

In [2]:
# Ignore cells with a borough that is Not assigned. 
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head(10)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Loop the unique list to repalce the Neighbourhood value

In [3]:
# combined different neighbourhood under the same boroough to the same row 
temp_list=df['Postcode'].unique().tolist() # Generate the unique list of the postcode
for i in temp_list:
    temp_count = df[df['Postcode']==i].shape[0] # Count how many records each postcode has
    if temp_count>1:
        index_list=df[df['Postcode']==i].index.tolist() # Get the index of these records wiith more than 1 record
        # Get all the unique nieghbourhood for each borough and then assign them to each borough
        df.loc[index_list,'Neighbourhood']=' ,'.join(df[df['Postcode']==i]['Neighbourhood'].unique().tolist())
        
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront ,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront ,Regent Park"
4,M6A,North York,"Lawrence Heights ,Lawrence Manor"
5,M6A,North York,"Lawrence Heights ,Lawrence Manor"
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge ,Malvern"
9,M1B,Scarborough,"Rouge ,Malvern"


### Allign the Neighbourhood value with borough

In [4]:
# If Neighbourhood is not assigned, then Neighourhood = borough

replace_list=df[df['Neighbourhood'] == "Not assigned"].index.tolist() # Find out the index for these Not assigned Neighbourhood
for index in replace_list:
    df.loc[index,'Neighbourhood']=df.loc[index,'Borough'] #for each index, use the Borough value to replae the "Not assigned"
    
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront ,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront ,Regent Park"
4,M6A,North York,"Lawrence Heights ,Lawrence Manor"
5,M6A,North York,"Lawrence Heights ,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge ,Malvern"
9,M1B,Scarborough,"Rouge ,Malvern"


### The number of rows of the final dataframe

In [5]:
df.shape[0]

211

# Part 2: Merge the Longitude and Latitude info

In [6]:
import os 

os.chdir("C:/Users/lxu/Desktop")

df_geo=pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [7]:
df_geo.rename(columns={"Postal Code":"Postcode"}, inplace=True)
df_geo.head(3)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [8]:
df=df.merge(df_geo, how="left", on='Postcode')
df.rename(columns={"Postcode": "PostalCode"},inplace=True)
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
3,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights ,Lawrence Manor",43.718518,-79.464763
5,M6A,North York,"Lawrence Heights ,Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
9,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353


# Part 3: Neighborhoods Analysis

In [9]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


For the bellowing analysis, I segmented the neighborhoods in Downtown Toronto only.

In [12]:
toronto_data=df[df['Borough'] == "Downtown Toronto"]
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
3,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
13,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937
14,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [13]:
address = 'Downtown Toronto, Toronto '

geolocator = Nominatim(user_agent="DT_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.654027, -79.3802003.


In [14]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = 'JMVDMC0MO0U3MKR5NL3SU3GFHT1TOLO4FYQE52DUHCCISMRW' # your Foursquare ID
CLIENT_SECRET = 'SBACRDKIXOBOXSYMYZIIKHKIJLXAR1LX2DAK3BX12KTNJQNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JMVDMC0MO0U3MKR5NL3SU3GFHT1TOLO4FYQE52DUHCCISMRW
CLIENT_SECRET:SBACRDKIXOBOXSYMYZIIKHKIJLXAR1LX2DAK3BX12KTNJQNK


In [16]:
toronto_data.reset_index(drop=True,inplace=True)

In [17]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront ,Regent Park are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Regent Part & Harbourfront within a radius of 500 meters.

In [19]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


#### Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Harbourfront ,Regent Park
Harbourfront ,Regent Park
Ryerson ,Garden District
Ryerson ,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide ,King ,Richmond
Adelaide ,King ,Richmond
Adelaide ,King ,Richmond
Harbourfront East ,Toronto Islands ,Union Station
Harbourfront East ,Toronto Islands ,Union Station
Harbourfront East ,Toronto Islands ,Union Station
Design Exchange ,Toronto Dominion Centre
Design Exchange ,Toronto Dominion Centre
Commerce Court ,Victoria Hotel
Commerce Court ,Victoria Hotel
Harbord ,University of Toronto
Harbord ,University of Toronto
Chinatown ,Grange Park ,Kensington Market
Chinatown ,Grange Park ,Kensington Market
Chinatown ,Grange Park ,Kensington Market
CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara
CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara
CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spa

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide ,King ,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.010000,0.0000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,King ,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.04
2                Café  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3             Airport  0.06
4  Airport Food Court  0.06


----Cabbagetown ,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2         Café  0.05
3       Bakery  0.05
4          Pub  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.13
1                Café  0.07
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4      Ic

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Restaurant,Steakhouse,Sushi Restaurant,Hotel,American Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Farmers Market,Beer Bar,Italian Restaurant,Seafood Restaurant
2,"CN Tower ,Bathurst Quay ,Island airport ,Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
3,"Cabbagetown ,St. James Town",Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Pharmacy,Market,Café,Grocery Store
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Ice Cream Shop,Burger Joint,Bar,Spa,Salad Place,Bubble Tea Shop


Run *k*-means to cluster the neighborhood into 5 clusters.

In [39]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 2, 0, 2, 4, 2, 0, 0])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Electronics Store
1,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Electronics Store
2,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Theater,Pizza Place,Burger Joint,Japanese Restaurant,Ice Cream Shop
3,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Theater,Pizza Place,Burger Joint,Japanese Restaurant,Ice Cream Shop
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Hotel,Restaurant,Café,Clothing Store,Bakery,Cosmetics Shop,Italian Restaurant,Breakfast Spot,Gastropub


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Electronics Store
1,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Electronics Store
6,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Ice Cream Shop,Burger Joint,Bar,Spa,Salad Place,Bubble Tea Shop
11,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Sporting Goods Shop,Music Venue
12,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Sporting Goods Shop,Music Venue
13,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Sporting Goods Shop,Music Venue
14,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Bar,Italian Restaurant
15,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Bar,Italian Restaurant
16,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Gym
17,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Gym


In [47]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,1,Park,Playground,Trail,Cosmetics Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [48]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Theater,Pizza Place,Burger Joint,Japanese Restaurant,Ice Cream Shop
3,Downtown Toronto,2,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Theater,Pizza Place,Burger Joint,Japanese Restaurant,Ice Cream Shop
4,Downtown Toronto,2,Coffee Shop,Hotel,Restaurant,Café,Clothing Store,Bakery,Cosmetics Shop,Italian Restaurant,Breakfast Spot,Gastropub
5,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Steakhouse,Farmers Market,Beer Bar,Italian Restaurant,Seafood Restaurant
8,Downtown Toronto,2,Coffee Shop,Café,Thai Restaurant,Bar,Restaurant,Steakhouse,Sushi Restaurant,Hotel,American Restaurant,Asian Restaurant
9,Downtown Toronto,2,Coffee Shop,Café,Thai Restaurant,Bar,Restaurant,Steakhouse,Sushi Restaurant,Hotel,American Restaurant,Asian Restaurant
10,Downtown Toronto,2,Coffee Shop,Café,Thai Restaurant,Bar,Restaurant,Steakhouse,Sushi Restaurant,Hotel,American Restaurant,Asian Restaurant
18,Downtown Toronto,2,Café,Bar,Restaurant,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Beer Bar,College Arts Building,Sandwich Place
19,Downtown Toronto,2,Café,Bar,Restaurant,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Beer Bar,College Arts Building,Sandwich Place
20,Downtown Toronto,2,Bar,Vietnamese Restaurant,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bakery,Dumpling Restaurant,Dessert Shop


In [49]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
24,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
25,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
26,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
27,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
28,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport
29,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court,Airport


In [50]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,4,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Convenience Store,Restaurant,Diner,Italian Restaurant
